In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scipy==1.1.0
!pip install emnist
# You need to restart the Runtime after installing the scipy

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from emnist import list_datasets
from sklearn import linear_model
list_datasets()


Using TensorFlow backend.


['balanced', 'byclass', 'bymerge', 'digits', 'letters', 'mnist']

In [ ]:
from emnist import extract_training_samples

X, y = extract_training_samples('byclass')
#X = np.array(X).reshape(-1, 28*28)
X = X[:10000]
y = y[:10000]

X = X/255.0
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
X_train = X_train.reshape((-1,784))
X_test = X_test.reshape((-1,784))

# Neural Network

In [ ]:
model = Sequential()
model.add(Dense(300, activation="relu", input_dim=784))
model.add(Dense(300, activation="relu"))
model.add(Dense(62, activation="softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

model.fit(
    X_train, to_categorical(y_train), epochs=15, batch_size=32
)

Epoch 1/15
219/219 [==============================] - 1s 4ms/step - loss: 1.8955 - accuracy: 0.5239
Epoch 2/15
219/219 [==============================] - 1s 4ms/step - loss: 1.0566 - accuracy: 0.6857
Epoch 3/15
219/219 [==============================] - 1s 4ms/step - loss: 0.8126 - accuracy: 0.7399
Epoch 4/15
219/219 [==============================] - 1s 4ms/step - loss: 0.6564 - accuracy: 0.7780
Epoch 5/15
219/219 [==============================] - 1s 4ms/step - loss: 0.5368 - accuracy: 0.8186
Epoch 6/15
219/219 [==============================] - 1s 4ms/step - loss: 0.4534 - accuracy: 0.8410
Epoch 7/15
219/219 [==============================] - 1s 4ms/step - loss: 0.3881 - accuracy: 0.8613
Epoch 8/15
219/219 [==============================] - 1s 4ms/step - loss: 0.3248 - accuracy: 0.8771
Epoch 9/15
219/219 [==============================] - 1s 4ms/step - loss: 0.2883 - accuracy: 0.8931
Epoch 10/15
219/219 [==============================] - 1s 4ms/step - loss: 0.2522 - accuracy: 0.9040

In [ ]:
#predictions = model.predict(X_test[:10])
#print(np.argmax(predictions, axis=1))
#print(y_test[:10])

#img = X_test[9].reshape((28,28))
#plt.imshow(img, cmap="gray")
#plt.show

In [ ]:
model.evaluate(X_test, to_categorical(y_test))

94/94 [==============================] - 0s 3ms/step - loss: 1.0945 - accuracy: 0.7220


[1.094527244567871, 0.722000002861023]

# KNN

In [ ]:
knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=7)
knn_clf.fit(X_train, y_train)

y_knn_pred = knn_clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_knn_pred)


0.6753333333333333

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(knn_clf, X_train, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

array([[227,   1,   0, ...,   0,   0,   0],
       [  0, 296,   0, ...,   0,   0,   0],
       [  7,   5, 282, ...,   0,   0,   1],
       ...,
       [  0,   2,   0, ...,  26,   0,   0],
       [  0,   7,   0, ...,   0,   1,   0],
       [  0,   1,  12, ...,   1,   0,   3]])

# SVM

In [ ]:
from sklearn.svm import SVC
model = SVC(C=10)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7406666666666667